# Título do Trabalho: Keras: Democratizando o Acesso ao Aprendizado Profundo
---
**Instituição:** Centro Universitário Carioca – UniCarioca
**Curso:** Engenharia da Computação  
**Disciplina:** Trabalho de Conclusão de Curso  
**Semestre:** 2025/2  
**Versão do Código:** 1.0 (Stable Release)  

**Equipe de Desenvolvimento:**
* **Adryel Salles Leite de Almeida** (Matrícula: 2020100122) – adryel7@gmail.com
* **Felipe S. de Lima** (Matrícula: 2021100794) – lima.felipesalles@gmail.com
* **Lucas C. Junker** (Matrícula: 2020102270) – lcsjunker@gmail.com

**Orientador(a):** Professor Ricardo Pires Mesquita   

---
### Objetivo

Demonstrar a viabilidade técnica e a simplicidade de implementação de Redes Neurais Artificiais (RNAs) utilizando o *framework* Keras, comprovando como ferramentas de alto nível democratizam o acesso ao Aprendizado Profundo (*Deep Learning*) para resolução de problemas de classificação de padrões.

---
### Metodologia

A presente pesquisa classifica-se como aplicada e quantitativa, adotando uma abordagem experimental para o desenvolvimento e validação do modelo de aprendizado de máquina. O procedimento metodológico foi estruturado em quatro etapas sequenciais:

**1. Ambiente de Desenvolvimento e Ferramentas:** O projeto foi desenvolvido na linguagem Python, utilizando o ambiente de execução em nuvem Google Colab, que fornece recursos computacionais necessários para o treinamento de redes neurais. As principais bibliotecas utilizadas foram:

**TensorFlow/Keras:** Para construção, compilação e treinamento da rede neural.

**Pandas e NumPy:** Para manipulação algébrica de dados e estruturação de logs.

**Matplotlib e Seaborn:** Para visualização de dados e geração de gráficos de desempenho.

**Scikit-learn:** Para cálculo de métricas avançadas de avaliação.

**2. Preparação e Pré-processamento dos Dados:** Utilizou-se o *dataset* MNIST (70.000 imagens de dígitos manuscritos). O *pipeline* de dados incluiu:

**Normalização:** Conversão dos valores de pixel da escala [0, 255] para [0, 1] para acelerar a convergência do gradiente.

**Definição de Sementes (*Seeds*):** Fixação de sementes aleatórias (seed=42) para garantir a reprodutibilidade científica dos experimentos.

***Data Augmentation*:** Aplicação de transformações dinâmicas (rotação, zoom e translação) durante o treinamento para aumentar a robustez do modelo contra variações espaciais.

**3. Arquitetura e Treinamento:** Foi definida uma arquitetura do tipo *Feedforward* (MLP) sequencial, composta por:

Camada de entrada (*Flatten*) para vetorização da imagem 28x28.

Camada oculta densa com 128 neurônios e função de ativação ReLU.

Camada de saída com 10 neurônios e função de ativação Softmax (distribuição de probabilidades). O treinamento utilizou o otimizador Adam e a função de perda *Sparse Categorical Crossentropy* ao longo de 15 épocas.

**4. Avaliação e Validação Prática:** A análise de resultados não se limitou à acurácia global. Foram gerados gráficos de histórico de treinamento para monitorar a convergência e uma Matriz de Confusão para identificar classes ambíguas. Por fim, realizou-se um Teste Prático de Inferência, submetendo o modelo a imagens externas desenhadas digitalmente, pré-processadas (inversão de cores e redimensionamento) para validar a eficácia da solução em ambiente de produção.



Bibliotecas

In [4]:
import os
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

ModuleNotFoundError: No module named 'matplotlib'

Reprodutibilidade e Definições de pastas

In [ ]:
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

PROJECT_VERSION = "1.0"

BASE_OUTPUTS = 'outputs'
BASE_MODELS = 'models'
BASE_DATA = os.path.join('data')

os.makedirs(BASE_OUTPUTS, exist_ok=True)
os.makedirs(BASE_MODELS, exist_ok=True)
os.makedirs(BASE_DATA, exist_ok=True)

print(f"--- Sistema Iniciado - Versão {PROJECT_VERSION} ---")
print("Ambiente configurado e sementes definidas.")

**Etapa 1 - Preparação dos dados**

Carregamento e preparação do *dataset* MNIST

In [ ]:
mnist=keras.datasets.mnist
(train_images, train_labels),(test_images, test_labels) = mnist.load_data()

Normalização dos pixels para o intervalo [0,1]

In [ ]:
train_images, test_images = train_images/255.0, test_images/255.0

*Reshape* para 4 dimensões

In [ ]:
train_images_aug = train_images.reshape(-1, 28, 28, 1)
test_images_aug = test_images.reshape(-1, 28, 28, 1)

**Etapa 2 - Definição da arquitetura**

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28,1)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

**Etapa 3 - Compilação**

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

**Etapa 4 - Treinamento**

Configurando *Data Augmentation*

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    fill_mode='constant',
    cval=0
)

Treinamento do modelo

In [ ]:
history = model.fit(
    datagen.flow(train_images_aug, train_labels, batch_size=32),
    epochs=15,
    validation_data=(test_images_aug, test_labels)
)

Salvando metadados do histórico de treinamento

In [ ]:
hist_df = pd.DataFrame(history.history)

hist_df['epoch'] = hist_df.index + 1

path_csv = os.path.join(BASE_OUTPUTS, 'training_log.csv')

hist_df.to_csv(path_csv, index=False)

print(f"Log de treinamento salvo em: {path_csv}")

print("\nPrévia dos dados salvos:")
print(hist_df.head())

Gráficos do treinamento

In [ ]:
path_csv = os.path.join(BASE_OUTPUTS, 'training_log.csv')
path_acc_graph = os.path.join(BASE_OUTPUTS, 'accuracy_plot.png')
path_loss_graph = os.path.join(BASE_OUTPUTS, 'loss_plot.png')

if not os.path.exists(path_csv):
    print(f"ERRO CRÍTICO: Arquivo de log não encontrado em {path_csv}")
else:
    print(f"Lendo histórico de: {path_csv}")
    df_log = pd.read_csv(path_csv)

Gráfico de Acurácia

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(df_log['epoch'], df_log['accuracy'], label='Treino', marker='o')
plt.plot(df_log['epoch'], df_log['val_accuracy'], label='Validação', marker='o')

plt.title('Evolução da Acurácia')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig(path_acc_graph, dpi=300)
plt.show()
print(f"Gráfico de Acurácia salvo em: {path_acc_graph}")

Gráfico de Perda(*Loss*)

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(df_log['epoch'], df_log['loss'], label='Treino', marker='o')
plt.plot(df_log['epoch'], df_log['val_loss'], label='Validação', marker='o')

plt.title('Evolução da Perda (Loss)')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig(path_loss_graph, dpi=300)
plt.show()
print(f"Gráfico de Perda salvo em: {path_loss_graph}")

**Etapa 5 - Avaliação de Desempenho**

Avaliando

In [ ]:
test_loss, test_acc = model.evaluate(test_images_aug, test_labels, verbose=0)

print(f"Acurácia Final: {test_acc * 100:.2f}%")
print(f"Perda Final: {test_loss:.4f}")

predictions = model.predict(test_images_aug)
class_predictions = np.argmax(predictions, axis=1)



Salvando metadados do histórico de Avaliação

In [ ]:
globals_data = {
    "acuracia_teste": test_acc,
    "perda_teste": test_loss,
    "total_amostras": len(test_images)
}
json_path = os.path.join(BASE_OUTPUTS, 'metrics.json')

print(f"Log salvo em: {json_path}")

Gerando Matriz de Confusão

In [ ]:
with open(json_path, 'w') as f:
    json.dump(globals_data, f, indent=4)

txt_report = classification_report(test_labels, class_predictions, digits=4)
path_txt = os.path.join(BASE_OUTPUTS, 'classification_report.txt')
with open(path_txt, 'w') as f:
    f.write(txt_report)

matrix = confusion_matrix(test_labels, class_predictions)

plt.figure(figsize=(10, 8))
sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusão - MNIST')
plt.ylabel('Rótulo Real')
plt.xlabel('Rótulo Predito')

path_matrix = os.path.join(BASE_OUTPUTS, 'matrix_confusao.png')
plt.savefig(path_matrix, dpi=300)
plt.show()
print(f"Dados do treinamento salvos em: {BASE_OUTPUTS}")

Salvando o modelo

In [ ]:
model_path = os.path.join(BASE_MODELS, 'mnist_model.keras')
model.save(model_path)
print(f"Modelo salvo em: {model_path}")

**Etapa 6 - Teste prático**


In [7]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

BASE_MODELS = 'models'
BASE_DATA = os.path.join('data')

imagepath = os.path.join(BASE_DATA, 'numberimg.png')

def predict_digit(filepath):
    if not os.path.exists(filepath):
        print(f"ERRO: Imagem não encontrada em: {filepath}")
        return

    print(f"\n--- Processando imagem: {filepath} ---")

    try:
        img = image.load_img(filepath, target_size=(28, 28), color_mode="grayscale")
        img_array = image.img_to_array(img)

        img_array = 255.0 - img_array

        img_array /= 255.0

        img_reshaped = img_array.reshape((1, 28, 28, 1))

        predicoes = model.predict(img_reshaped)
        predict_class = np.argmax(predicoes)
        predict_confidence = np.max(predicoes) * 100


        plt.figure(figsize=(4, 4))
        plt.imshow(img_array.reshape(28, 28), cmap='gray')
        plt.title(f"Eu vejo o número: {predict_class}\nCerteza: {predict_confidence:.2f}%")
        plt.axis('off')
        plt.show()

        return predict_class

    except Exception as e:
        print(f"Ocorreu um erro ao processar a imagem: {e}")

predict_digit(imagepath)

ModuleNotFoundError: No module named 'tensorflow'

---
## 7. Conclusão e Considerações Finais

O desenvolvimento deste projeto permitiu demonstrar a eficácia do *framework* **Keras** na implementação ágil de redes neurais profundas. Através da construção de uma arquitetura *Multilayer Perceptron* (MLP), foi possível atingir uma acurácia competitiva no dataset MNIST.

**Principais Resultados Alcançados:**
* **Robustez:** A aplicação de *Data Augmentation* mitigou o *overfitting*, permitindo que o modelo generalizasse bem para dados de teste (Acurácia > 97%) e mantivesse a consistência entre as curvas de treino e validação.
* **Aplicabilidade Prática:** O teste de inferência com imagens externas comprovou que o modelo não apenas memorizou o *dataset*, mas aprendeu a identificar padrões estruturais dos dígitos, funcionando corretamente com *inputs* do mundo real.
* **Análise de Erros:** A Matriz de Confusão permitiu identificar que as maiores dificuldades do modelo residem em ambiguidades topológicas, fornecendo um caminho claro para melhorias futuras.

**Trabalhos Futuros:**
Visando a evolução deste protótipo acadêmico para um produto de *software* utilizável (*User-Friendly*), propõe-se:

1.  **Desenvolvimento de Interface Web Interativa (Web App):**
    A implementação de uma interface utilizando *frameworks* como **Streamlit** ou **Gradio**. Tais ferramentas permitem a integração de um *Canvas* digital (painel de desenho), onde o usuário pode desenhar o dígito diretamente no navegador com o mouse ou dedo, eliminando a necessidade de *softwares* externos (como Paint) e *upload* de arquivos.

2.  **Implementação de Arquitetura CNN:**
    Substituir a atual rede MLP por **Redes Neurais Convolucionais (CNNs)**. Esta arquitetura preserva a estrutura espacial 2D da imagem, sendo o estado da arte para visão computacional, o que elevaria a acurácia para níveis superiores a 99% e reduziria erros de confusão geométrica.

3.  **Disponibilização via API (Microserviço):**
    O encapsulamento do modelo treinado em uma **API REST** (utilizando FastAPI ou Flask). Isso permitiria que o modelo fosse consumido por aplicações externas, como aplicativos móveis (Android/iOS), permitindo o reconhecimento de dígitos a partir da câmera do celular em tempo real.

**Entregáveis:**
Todos os artefatos gerados (modelo treinado `.keras`, gráficos de desempenho e logs) encontram-se salvos e organizados nos diretórios `models/` e `outputs/`.

In [ ]:
import shutil
from google.colab import files

print("--- Empacotando resultados para download ---")

shutil.make_archive('resultados_tcc', 'zip', BASE_OUTPUTS)

shutil.make_archive('modelo_tcc', 'zip', BASE_MODELS)

print("Arquivos zipados com sucesso!")
print("Iniciando download...")

files.download('resultados_tcc.zip')
files.download('modelo_tcc.zip')